<a href="https://colab.research.google.com/github/imogenagle/CHIMERA-New/blob/main/CHIMERA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import astropy
from astropy import wcs
from astropy.io import fits
from astropy.modeling.models import Gaussian2D
import astropy.units as u
from astropy.utils.data import download_file
from astropy.visualization import astropy_mpl_style
import cv2
import glob
import mahotas
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.interpolate
import sunpy
import sunpy.map
import sys

plt.style.use(astropy_mpl_style)

In [ ]:
im171 = glob.glob('171.fts')
im193 = glob.glob('193.fts')
im211 = glob.glob('211.fts')
imhmi = glob.glob('hmi.fts')

In [ ]:
if im171 == [] or im193 == [] or im211 == [] or imhmi == []:
	print("Not all required files present")
	sys.exit()

In [ ]:
x=np.arange(0,1024)*4
hdu_number=0
heda=fits.getheader(im171[0],hdu_number)
data= fits.getdata(im171[0], ext=0)/(heda["EXPTIME"])
dn=scipy.interpolate.interp2d(x,x,data)
data=dn(np.arange(0,4096),np.arange(0,4096))

In [ ]:
hedb=fits.getheader(im193[0],hdu_number)
datb= fits.getdata(im193[0], ext=0)/(hedb["EXPTIME"])
dn=scipy.interpolate.interp2d(x,x,datb)
datb=dn(np.arange(0,4096),np.arange(0,4096))

In [ ]:
hedc=fits.getheader(im211[0],hdu_number)
datc= fits.getdata(im211[0], ext=0)/(hedc["EXPTIME"])
dn=scipy.interpolate.interp2d(x,x,datc)
datc=dn(np.arange(0,4096),np.arange(0,4096))

In [ ]:
hedm=fits.getheader(imhmi[0],hdu_number)
datm= fits.getdata(imhmi[0], ext=0)

In [ ]:
if hedm['crota1'] > 90:
	datm=np.rot90(np.rot90(datm))

In [ ]:
s=np.shape(data)
rs=heda['rsun']

In [ ]:
if hedb["ctype1"] != 'solar_x ':
	hedb["ctype1"]='solar_x '
	hedb["ctype2"]='solar_y '

In [ ]:
if heda['cdelt1'] > 1:
	heda['cdelt1'],heda['cdelt2'],heda['crpix1'],heda['crpix2']=heda['cdelt1']/4.,heda['cdelt2']/4.,heda['crpix1']*4.0,heda['crpix2']*4.0
	hedb['cdelt1'],hedb['cdelt2'],hedb['crpix1'],hedb['crpix2']=hedb['cdelt1']/4.,hedb['cdelt2']/4.,hedb['crpix1']*4.0,hedb['crpix2']*4.0
	hedc['cdelt1'],hedc['cdelt2'],hedc['crpix1'],hedc['crpix2']=hedc['cdelt1']/4.,hedc['cdelt2']/4.,hedc['crpix1']*4.0,hedc['crpix2']*4.0

In [ ]:
dattoarc=heda['cdelt1']
conver=(s[0]/2)*dattoarc/hedm['cdelt1']-(s[1]/2)
convermul=dattoarc/hedm['cdelt1']

In [ ]:
aia=sunpy.map.Map(im171)
adj=4096./aia.dimensions[0].value
x, y = (np.meshgrid(*[np.arange(adj*v.value) for v in aia.dimensions]) * u.pixel)/adj
hpc = aia.pixel_to_world(x, y)
hg=hpc.transform_to(sunpy.coordinates.frames.HeliographicStonyhurst)

In [ ]:
csys=wcs.WCS(hedb)

In [ ]:
ident=1
iarr=np.zeros((s[0],s[1]),dtype=np.byte)
offarr,slate=np.array(iarr),np.array(iarr)
bmcool=np.zeros((s[0],s[1]),dtype=np.float32)
cand,bmmix,bmhot=np.array(bmcool),np.array(bmcool),np.array(bmcool)
circ=np.zeros((s[0],s[1]),dtype=int)

In [ ]:
r = (s[1]/2.0)-450
xgrid,ygrid=np.meshgrid(np.arange(s[0]),np.arange(s[1]))
center=[int(s[1]/2.0),int(s[1]/2.0)]
w=np.where((xgrid-center[0])**2+(ygrid-center[1])**2 > r**2)
y,x=np.mgrid[0:4096,0:4096]
garr=Gaussian2D(1,s[0]/2,s[1]/2,2000/2.3548,2000/2.3548)(x,y)
garr[w]=1.0

In [ ]:
props=np.zeros((26,30),dtype='<U16')
props[:,0]='ID','XCEN','YCEN','CENTROID','X_EB','Y_EB','X_WB','Y_WB','X_NB','Y_NB','X_SB','Y_SB','WIDTH','WIDTH°','AREA','AREA%','<B>','<B+>','<B->','BMAX','BMIN','TOT_B+','TOT_B-','<PHI>','<PHI+>','<PHI->'
props[:,1]='num','"','"','H°','"','"','"','"','"','"','"','"','H°','°','Mm^2','%','G','G','G','G','G','G','G','Mx','Mx','Mx'

In [ ]:
data[np.where(data <= 0)]=0
datb[np.where(datb <= 0)]=0
datc[np.where(datc <= 0)]=0

In [ ]:
with np.errstate(divide = 'ignore'):
	t0=np.log10(datc)
	t1=np.log10(datb)
	t2=np.log10(data)

In [ ]:
t0[np.where(t0 < 0.8)] = 0.8
t0[np.where(t0 > 2.7)] = 2.7
t1[np.where(t1 < 1.4)] = 1.4
t1[np.where(t1 > 3.0)] = 3.0
t2[np.where(t2 < 1.2)] = 1.2
t2[np.where(t2 > 3.9)] = 3.9

In [ ]:
t0=np.array(((t0-0.8)/(2.7-0.8))*255,dtype=np.float32)
t1=np.array(((t1-1.4)/(3.0-1.4))*255,dtype=np.float32)
t2=np.array(((t2-1.2)/(3.9-1.2))*255,dtype=np.float32)

In [ ]:
with np.errstate(divide = 'ignore',invalid='ignore'):
	bmmix[np.where(t2/t0 >= ((np.mean(data)*0.6357)/(np.mean(datc))))]=1
	bmhot[np.where(t0+t1 < (0.7*(np.mean(datb)+np.mean(datc))))]=1
	bmcool[np.where(t2/t1 >= ((np.mean(data)*1.5102)/(np.mean(datb))))]=1

In [ ]:
cand=bmcool*bmmix*bmhot

In [ ]:
r=(s[1]/2.0)-100
w=np.where((xgrid-center[0])**2+(ygrid-center[1])**2 <= r**2)
circ[w]=1.0
cand=cand*circ

In [ ]:
circ[:]=0
r=(rs/dattoarc)-10
w=np.where((xgrid-center[0])**2+(ygrid-center[1])**2 <= r**2)
circ[w]=1.0
r=(rs/dattoarc)+40
w=np.where((xgrid-center[0])**2+(ygrid-center[1])**2 >= r**2)
circ[w]=1.0
cand=cand*circ

In [ ]:
cand=np.array(cand,dtype=np.uint8)
cont,heir=cv2.findContours(cand,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
sizes=[]
for i in range(len(cont)):
	sizes=np.append(sizes,len(cont[i]))

In [ ]:
reord=sizes.ravel().argsort()[::-1]

In [ ]:
tmp=list(cont)

In [ ]:
for i in range(len(cont)):
	tmp[i]=cont[reord[i]]

In [ ]:
cont=list(tmp)

In [ ]:
for i in range(len(cont)):

	x=np.append(x,len(cont[i]))